In [1]:
%load_ext autoreload
%autoreload 2

In [51]:
import json
import sys
import re
import os
from pathlib import Path
from functools import partial

import blpapi
import clarion
import ipywidgets as w
import ipyvuetify as v
import numpy as np
import pandas as pd
import QuantLib as ql
from IPython.display import display
from ipydatagrid import DataGrid, TextRenderer, BarRenderer, Expr
from bqplot import LinearScale, ColorScale, OrdinalColorScale, OrdinalScale
from pandas.tseries.offsets import BDay

import clarion as cl
import bhutils
from bhutils import bhutilspy as bh
from api.ezutils import bh, mkt_defs, ccy_cals, mx_cals, mtg_dts, gens

from api.utils import parse_offset

In [3]:
curves_df = pd.DataFrame(
    [
        ('AUD', 'AUD.6M.STD', 'AUD.D.AUDC'),
        ('CAD', 'CAD.3M.STD', 'CAD.D.CADC'),
        ('CHF', 'CHF.6M.STD', 'CHF.D.CHFC'),
        ('EUR', 'EUR.6M.STD', 'EUR.D.EURC'),
        ('GBP', 'GBP.6M.STD', 'GBP.D.GBPC'),
        ('NOK', 'NOK.6M.STD', 'NOK.D.NOKC'),
        ('NZD', 'NZD.3M.STD', 'NZD.D.NZDC'),
        ('SEK', 'SEK.6M.STD', 'SEK.D.SEKC'),
        ('USD', 'USD.SOFR', 'USD.D'),
        ('JPY', 'JPY.6M.STD', 'JPY.D.JPYC'),
    ],
    columns=('ccy', 'std', 'disc')
).set_index('ccy')

In [43]:
today = pd.Timestamp.today().floor('d')
chg = 1
close = today - BDay(chg)

In [85]:
class LiveCurvesLocal:
    def __init__(self, date=None, curves=None):
        self.date = date or pd.Timestamp.today().floor('d')
        self.curves = curves
        self.sheet_mkt = bh.bhCreateSheetMarket(
            _handle='sheet_mkt',
            _mkt='LIVE',
            _asof=self.date,
            _mktdefs=mkt_defs,
            _calprov=mx_cals,
            _mtgdts=mtg_dts,
            _crvdbsvr="SQLP1-A",
            _crvdbname="Curves",
            _mktdatasvr="localhost",
            _mktdataport=60000,
            _snapdbsvr="SQLP1-B",
            _snapdbname="Snap",
        )
    
    @property
    def mkt_snap(self):
        return bh.bhCurveMarketSnap(
            _handle='crv_mkt_snap',
            _market=self.sheet_mkt,
            _curves=self.curves
        )
    
    @property
    def mkt_snap_db(self):
        return bh.bhCurveMarketSnapAsDb(
            _handle='crv_mkt_snap_db',
            _marketsnap=self.mkt_snap
        )
    
    @property
    def mkt_snap_db_curves(self):
        return bh.bhDbLookUp(
            _rsid=self.mkt_snap_db,
            _lookupvalues="LIVE",
            _lookupcols="name",
            _returncols="curves"
        )
    
    @property
    def db_curves(self):
        return self.mkt_snap_db_curves
    
    
class COBCurvesRemote:
    def __init__(self, date=None, curves=None, offset=None):
        self.date = date or pd.Timestamp.today().floor('d') - BDay()
        if offset:
            self.date = pd.Timestamp.today().floor('d') - parse_offset(offset)
        self.curves = curves
        
        self.db_mkt = bh.bhCreateCurvesDbMarket(
            _handle=f'crv_db_mkt_{self.date}',
            _mktname="COB",
            _asofdate=self.date,
            _datadate=self.date,
            _mktdefs=mkt_defs,
            _cals=mx_cals,
            _curvesserver="SQLP1-A",
            _curvesdb="Curves",
            _minextraptnr='80y'
        )
        
        self.mkt_snap_cob = bh.bhMarketSnap(
            _handle=f'mkt_snap_cob_{self.date:%Y%m%d}',
            _market=self.db_mkt,
            _curves=self.curves
        )
        
        self.db_curves = self.mkt_snap_cob

In [245]:
from api.data.base import radar_path

In [246]:
radar_path.exists()

True

In [248]:
data = data.dropna(subset='ccy')

In [249]:
data.to_csv(radar_path / 'save/test.csv', index=False)

In [201]:
live_curves = LiveCurvesLocal(curves=curves_df.melt()['value'].to_list())
cob_curves = COBCurvesRemote(curves=curves_df.melt()['value'].to_list())

### LIVE Curves (local)

In [108]:
sheet_mkt = bh.bhCreateSheetMarket(
    _handle='sheet_mkt',
    _mkt='LIVE',
    _asof=today,
    _mktdefs=mkt_defs,
    _calprov=mx_cals,
    _mtgdts=mtg_dts,
    _crvdbsvr="SQLP1-A",
    _crvdbname="Curves",
    _mktdatasvr="localhost",
    _mktdataport=60000,
    _snapdbsvr="SQLP1-B",
    _snapdbname="Snap",
)

crv_mkt_snap= bh.bhCurveMarketSnap(
    _handle='crv_mkt_snap',
    _market=sheet_mkt,
    _curves=curves_df.melt()['value'].to_list()
)

crv_mkt_snap_db = bh.bhCurveMarketSnapAsDb(
    _handle='crv_mkt_snap_db',
    _marketsnap=crv_mkt_snap
)

crv_mkt_snap_db_curves = bh.bhDbLookUp(
    _rsid=crv_mkt_snap_db,
    _lookupvalues="LIVE",
    _lookupcols="name",
    _returncols="curves"
)

### COB Curves  

In [45]:
crv_db_mkt = bh.bhCreateCurvesDbMarket(
    _handle=f'crv_db_mkt_{chg}',
    _mktname="COB",
    _asofdate=close,
    _datadate=close,
    _mktdefs=mkt_defs,
    _cals=mx_cals,
    _curvesserver="SQLP1-A",
    _curvesdb="Curves",
    _minextraptnr='80y'
)

mkt_snap_cob = bh.bhMarketSnap(
    _handle=f'mkt_snap_cob_{chg}',
    _market=crv_db_mkt,
    _curves=curves_df.melt()['value'].to_list()
)

In [46]:
ccy = 'EUR'
tenor = '8yx2y'

In [91]:
def get_sw_rate_from_ccy(
    ccy, 
    tenor, 
    curves,
    curves_df
):
    disc_ = bh.bhDbLookUp(
        _rsid=curves.db_curves, 
        _lookupvalues=curves_df.loc[ccy, 'disc'],
        _lookupcols="name",
        _returncols="curve",
        _matchtype=0
    )
    
    proj_ = bh.bhDbLookUp(
        _rsid=curves.db_curves, 
        _lookupvalues=curves_df.loc[ccy, 'std'],
        _lookupcols="name",
        _returncols="curve",
        _matchtype=0
    )
    
    rate = bh.bhRate2(
        _crv=disc_,
        _mtyortenor=tenor,
        _type='sw',
        _name=curves_df.loc[ccy, 'std'],
        _mktdefs=mkt_defs,
        _calprov=mx_cals,
        _leavemtyflag=1,
        _mul=100,
        _projcurve=proj_,
        _mtgdates=mtg_dts
    )
    return rate


In [118]:
ex = [
    {'ccy': 'EUR', 'tenor': '5yx1y'},
    {'ccy': 'EUR', 'tenor': '10yx2y'},
    {'ccy': 'USD', 'tenor': '18mx1y'},
    {'ccy': 'USD', 'tenor': '5yx1y'},
    {'ccy': 'AUD', 'tenor': '2yx2y'},
    {'ccy': 'USD', 'tenor': '2yx2y'},
    {'ccy': 'AUD', 'tenor': '6mx1y'},
    {'ccy': 'USD', 'tenor': '6mx1y'},
    {'ccy': 'EUR', 'tenor': '11mx9y'},
    {'ccy': 'USD', 'tenor': '11mx9y'},
    {'ccy': 'USD', 'tenor': '7mx1y'},
    {'ccy': 'USD', 'tenor': '4yx1y'},
    {'ccy': 'USD', 'tenor': '3yx2y'},
    {'ccy': 'USD', 'tenor': '7mx1y'},
    {'ccy': 'AUD', 'tenor': '7mx1y'},
]

ex = pd.DataFrame(ex)

In [171]:
pd.read_clipboard().to_dict('records')

[{'name': 'EURF1',
  'ccy': 'EUR',
  'wgt1': -0.85,
  'leg1': '5yx1y',
  'wgt2': 2,
  'leg2': '6yx2y',
  'wgt3': -1.15,
  'leg3': '8yx2y'},
 {'name': 'EURF2',
  'ccy': 'EUR',
  'wgt1': -0.85,
  'leg1': '10yx2y',
  'wgt2': 2,
  'leg2': '12yx3y',
  'wgt3': -1.15,
  'leg3': '15yx5y'},
 {'name': 'USD FLY1',
  'ccy': 'USD',
  'wgt1': -1.0,
  'leg1': '18mx1y',
  'wgt2': 2,
  'leg2': '30mx1y',
  'wgt3': -1.0,
  'leg3': '4yx1y'},
 {'name': 'USD FLY2',
  'ccy': 'USD',
  'wgt1': -1.0,
  'leg1': '5yx1y',
  'wgt2': 2,
  'leg2': '6yx2y',
  'wgt3': -1.0,
  'leg3': '8yx2y'},
 {'name': 'AUD Curve',
  'ccy': 'AUD',
  'wgt1': -1.0,
  'leg1': '2yx2y',
  'wgt2': 1,
  'leg2': '10yx5y',
  'wgt3': nan,
  'leg3': nan},
 {'name': 'USD Curve',
  'ccy': 'USD',
  'wgt1': -1.0,
  'leg1': '2yx2y',
  'wgt2': 1,
  'leg2': '10yx5y',
  'wgt3': nan,
  'leg3': nan},
 {'name': 'AUD FLYX',
  'ccy': 'AUD',
  'wgt1': -1.0,
  'leg1': '6mx1y',
  'wgt2': 2,
  'leg2': '18mx1y',
  'wgt3': -1.0,
  'leg3': '3yx4y'},
 {'name': 'USD 

In [197]:
from numpy import nan
data = pd.DataFrame(
    [{'name': 'EURF1',
  'ccy': 'EUR',
  'wgt1': -0.85,
  'leg1': '5yx1y',
  'wgt2': 2,
  'leg2': '6yx2y',
  'wgt3': -1.15,
  'leg3': '8yx2y'},
 {'name': 'EURF2',
  'ccy': 'EUR',
  'wgt1': -0.85,
  'leg1': '10yx2y',
  'wgt2': 2,
  'leg2': '12yx3y',
  'wgt3': -1.15,
  'leg3': '15yx5y'},
 {'name': 'USD FLY1',
  'ccy': 'USD',
  'wgt1': -1.0,
  'leg1': '18mx1y',
  'wgt2': 2,
  'leg2': '30mx1y',
  'wgt3': -1.0,
  'leg3': '4yx1y'},
 {'name': 'USD FLY2',
  'ccy': 'USD',
  'wgt1': -1.0,
  'leg1': '5yx1y',
  'wgt2': 2,
  'leg2': '6yx2y',
  'wgt3': -1.0,
  'leg3': '8yx2y'},
 {'name': 'AUD Curve',
  'ccy': 'AUD',
  'wgt1': -1.0,
  'leg1': '2yx2y',
  'wgt2': 1,
  'leg2': '10yx5y',
  'wgt3': nan,
  'leg3': nan},
 {'name': 'USD Curve',
  'ccy': 'USD',
  'wgt1': -1.0,
  'leg1': '2yx2y',
  'wgt2': 1,
  'leg2': '10yx5y',
  'wgt3': nan,
  'leg3': nan},
 {'name': 'AUD FLYX',
  'ccy': 'AUD',
  'wgt1': -1.0,
  'leg1': '6mx1y',
  'wgt2': 2,
  'leg2': '18mx1y',
  'wgt3': -1.0,
  'leg3': '3yx4y'},
 {'name': 'USD FLYX',
  'ccy': 'USD',
  'wgt1': -1.0,
  'leg1': '6mx1y',
  'wgt2': 2,
  'leg2': '18mx1y',
  'wgt3': -1.0,
  'leg3': '3yx4y'},
 {'name': 'EUR Curve US',
  'ccy': 'EUR',
  'wgt1': -1.0,
  'leg1': '11mx9y',
  'wgt2': 1,
  'leg2': '20yx10y',
  'wgt3': nan,
  'leg3': nan},
 {'name': 'USD Curve EUR',
  'ccy': 'USD',
  'wgt1': -1.0,
  'leg1': '11mx9y',
  'wgt2': 1,
  'leg2': '20yx10y',
  'wgt3': nan,
  'leg3': nan},
 {'name': 'US FLY1',
  'ccy': 'USD',
  'wgt1': -1.15,
  'leg1': '7mx1y',
  'wgt2': 2,
  'leg2': '19mx1y',
  'wgt3': -0.85,
  'leg3': '37mx1y'},
 {'name': 'US FLY2',
  'ccy': 'USD',
  'wgt1': -1.15,
  'leg1': '4yx1y',
  'wgt2': 2,
  'leg2': '6yx4y',
  'wgt3': -0.85,
  'leg3': '15yx5y'},
 {'name': 'US Curve',
  'ccy': 'USD',
  'wgt1': -1.0,
  'leg1': '3yx2y',
  'wgt2': 1,
  'leg2': '5yx2y',
  'wgt3': nan,
  'leg3': nan},
 {'name': 'US FLY3',
  'ccy': 'USD',
  'wgt1': -0.85,
  'leg1': '7mx1y',
  'wgt2': 2,
  'leg2': '19mx2y',
  'wgt3': -1.15,
  'leg3': '61mx1y'},
 {'name': 'AUD FLY3',
  'ccy': 'AUD',
  'wgt1': -0.85,
  'leg1': '7mx1y',
  'wgt2': 2,
  'leg2': '19mx2y',
  'wgt3': -1.15,
  'leg3': '61mx1y'},
 {'name': 'USDGBPxFLY',
  'ccy': 'USD',
  'wgt1': -0.8,
  'leg1': '10mx1y',
  'wgt2': 2,
  'leg2': '22mx2y',
  'wgt3': -1.2,
  'leg3': '58mx2y'},
 {'name': 'GBPUSDxFLY',
  'ccy': 'GBP',
  'wgt1': -0.8,
  'leg1': '10mx1y',
  'wgt2': 2,
  'leg2': '22mx2y',
  'wgt3': -1.2,
  'leg3': '58mx2y'}]
                 )

In [202]:
def parse_params_df(data):
    df = data

    params = []
    for index, row in df.iterrows():
        name, ccy = row['name'], row['ccy']
        row = row.dropna()
        row_sub = row[2:]
        grouper = [l[0] for l in row_sub.index.str.findall('\d+')]
        g = row_sub.groupby(grouper, sort=False)
        row_params = []
        for index, group in g:
            param = {'name': name, 'ccy': ccy, 'tenor': group[f'leg{index}'], 'wgt': group[f'wgt{index}'], 'group': index}
            row_params.append(param)

        params.extend(row_params)
    return params

In [232]:
def update_flies_data(data):
    params = parse_params_df(data)
    flat_data = pd.DataFrame(params)
    flat_data = flat_data.assign(
        rate=get_sw_rate_from_params(params, live_curves, curves_df),
        cob=get_sw_rate_from_params(params, cob_curves, curves_df)
    )
    
    new_rows = []
    g = flat_data.groupby('name', sort=False)
    for index, group in g:
        group['group'] = group['group'].astype(int)
        group = group.sort_values('group')
        new_row = {'name': group.iloc[0]['name'], 'ccy': group.iloc[0]['ccy']}

        for _, row in group.iterrows():
            new_row[f"wgt{row['group']}"] = row['wgt']
            new_row[f"leg{row['group']}"] = row['tenor']

        for _, row in group.iterrows():
            new_row[f"rate{row['group']}"] = row['rate']
        new_row['live'] = group['wgt'] @ group['rate']

        for _, row in group.iterrows():    
            new_row[f"change{row['group']}"] = row['rate'] - row['cob']
        new_row['change'] = group['wgt'] @ (row['rate'] - group['cob'])
        new_rows.append(new_row)
    return pd.DataFrame(new_rows)

In [198]:
params = parse_params_df(data)

In [233]:
update_flies_data(data)

name  ccy  wgt1    leg1  wgt2     leg2  wgt3    leg3     rate1  \
0           EURF1  EUR -0.85   5yx1y   2.0    6yx2y -1.15   8yx2y  3.154901   
1           EURF2  EUR -0.85  10yx2y   2.0   12yx3y -1.15  15yx5y  3.145703   
2        USD FLY1  USD -1.00  18mx1y   2.0   30mx1y -1.00   4yx1y  3.881104   
3        USD FLY2  USD -1.00   5yx1y   2.0    6yx2y -1.00   8yx2y  3.406240   
4       AUD Curve  AUD -1.00   2yx2y   1.0   10yx5y   NaN     NaN  4.348250   
5       USD Curve  USD -1.00   2yx2y   1.0   10yx5y   NaN     NaN  3.579107   
6        AUD FLYX  AUD -1.00   6mx1y   2.0   18mx1y -1.00   3yx4y  4.596272   
7        USD FLYX  USD -1.00   6mx1y   2.0   18mx1y -1.00   3yx4y  4.389853   
8    EUR Curve US  EUR -1.00  11mx9y   1.0  20yx10y   NaN     NaN  3.227209   
9   USD Curve EUR  USD -1.00  11mx9y   1.0  20yx10y   NaN     NaN  3.537290   
10        US FLY1  USD -1.15   7mx1y   2.0   19mx1y -0.85  37mx1y  4.365105   
11        US FLY2  USD -1.15   4yx1y   2.0    6yx4y -0.85  15yx5y  3.429141   
12       US Curve  USD -1.00   3yx2y   1.0    5yx2y   NaN     NaN  3.461759   
13        US FLY3  USD -0.85   7mx1y   2.0   19mx2y -1.15  61mx1y  4.365105   
14       AUD FLY3  AUD -0.85   7mx1y   2.0   19mx2y -1.15  61mx1y  4.602420   
15     USDGBPxFLY  USD -0.80  10mx1y   2.0   22mx2y -1.20  58mx2y  4.255087   
16     GBPUSDxFLY  GBP -0.80  10mx1y   2.0   22mx2y -1.20  58mx2y  6.404683   

       rate2     rate3      live   change1   change2   change3    change  
0   3.179863  3.291987 -0.107725  0.082019  0.120060  0.132163  0.126140  
1   2.821876  1.852092  0.839997  0.115574  0.107864  0.026285 -0.752736  
2   3.551544  3.429141 -0.207157 -0.129890 -0.089716 -0.054745  0.212361  
3   3.350935  3.358258 -0.062629 -0.001916  0.018647  0.016123  0.085716  
4   4.706710       NaN  0.358460  0.008824  0.120316       NaN -0.246967  
5   3.310011       NaN -0.269097 -0.095227  0.028406       NaN  0.392730  
6   4.408399  4.620900 -0.400375 -0.074994 -0.022796  0.078587  0.351190  
7   3.881104  3.424315 -0.051960 -0.189863 -0.129890 -0.034128  0.016170  
8   1.054617       NaN -2.172591  0.062417 -0.016434       NaN  2.093740  
9   2.082981       NaN -1.454309 -0.045955 -0.009892       NaN  1.490372  
10  3.832186  3.484729 -0.317519 -0.188115 -0.128189 -0.081582  0.346817  
11  3.354475  2.835514  0.355252 -0.054745  0.017426 -0.005142 -0.253074  
12  3.384194       NaN -0.077565 -0.068771  0.002911       NaN  0.149247  
13  3.688428  3.403623 -0.247649 -0.188115 -0.108668  0.000550  0.189578  
14  4.359365  4.763973 -0.671897 -0.075215 -0.005796  0.112143  0.595273  
15  3.616555  3.389866 -0.238798 -0.175479 -0.101149 -0.001397  0.178559  
16  5.494614  3.728900  1.390802  0.095798  0.186823 -0.355143 -0.667621

In [234]:
data = pd.read_clipboard()

In [237]:
data = data.dropna(how='all')

In [240]:
data = data.drop_duplicates(subset='name')

In [123]:
def get_sw_rate_from_params(
    params: [list, pd.DataFrame], 
    curves,
    curves_df
):
    
    '''
    param can be a list of dictionaries with keys 'ccy' and tenor or a dataframe with columns ccy and tenor 
    '''
    
    if isinstance(params, list):
        params = pd.DataFrame(params)
    db_curves = curves.db_curves
    rates = [get_sw_rate_from_ccy(row['ccy'], row['tenor'], db_curves=db_curves, curves_df=curves_df) for index, row in params.iterrows()]
    return rates

In [120]:
def get_sw_rate_from_ccy(
    ccy,
    tenor,
    db_curves,
    curves_df
):
    
    disc_ = bh.bhDbLookUp(
        _rsid=db_curves, 
        _lookupvalues=curves_df.loc[ccy, 'disc'],
        _lookupcols="name",
        _returncols="curve",
        _matchtype=0
    )
    
    proj_ = bh.bhDbLookUp(
        _rsid=db_curves, 
        _lookupvalues=curves_df.loc[ccy, 'std'],
        _lookupcols="name",
        _returncols="curve",
        _matchtype=0
    )
    
    rate = bh.bhRate2(
        _crv=disc_,
        _mtyortenor=tenor,
        _type='sw',
        _name=curves_df.loc[ccy, 'std'],
        _mktdefs=mkt_defs,
        _calprov=mx_cals,
        _leavemtyflag=1,
        _mul=100,
        _projcurve=proj_,
        _mtgdates=mtg_dts
    )
    return rate


In [98]:
def get_sw_rate_from_generator(
    generator, 
    tenor, 
    curves,
    collateral='LCH'
):
    rate = bh.bhFiRate(
        _efforfwd=tenor.lower().split('x')[0],
        _mtyortenor=tenor.lower().split('x')[1],
        _generatorname=generator,
        _curves=curves.db_curves,
        _collateral=collateral,
        _generators=gens
    )
    return rate


In [99]:
gen = 'EURIBOR A 6M*'

In [110]:
r1_live = get_sw_rate_from_ccy(
    ccy=ccy,
    tenor=tenor,
    curves=live_curves,
    curves_df=curves_df
)
r1_live

3.2991118188241964

In [100]:
r1_cob = get_sw_rate_from_ccy(
    ccy=ccy,
    tenor=tenor,
    curves=close_curves,
    curves_df=curves_df
)

In [111]:
get_sw_rate_from_generator(
    generator=gen,
    tenor=tenor,
    curves=live_curves,
)

0.0330004411061224

In [59]:
bh.bhFiRate(
    _efforfwd=tenor.lower().split('x')[0],
    _mtyortenor=tenor.lower().split('x')[1],
    _generatorname='EURIBOR A 6M*',
    _curves=crv_mkt_snap_db_curves,
    _collateral='LCH',
    _generators=gens
)

0.031853364961682944

In [58]:
r1_live

3.2344939309584557

In [50]:
r1_live - r1_cob

0.07466937090266512

In [ ]:
bh.bhFiRate(
    


)

In [4]:
input_cols = [
    'name',
    'ccy',
    'wgt1',
    'leg1',
    'wgt2',
    'leg2',
    'wgt3',
    'leg3',
]